# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
file = "../output_data/city_csv"

city_df = pd.read_csv(file)
city_df = city_df.drop(columns="Unnamed: 0")
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ibrā’,22.69,58.53,75.09,43,68,7.36,OM,1605847912
1,Kirakira,-10.45,161.92,79.79,86,100,10.78,SB,1605847679
2,Bilibino,68.05,166.44,8.91,88,95,0.38,RU,1605847913
3,Kapaa,22.08,-159.32,77.00,78,40,14.99,US,1605847913
4,Xinqing,48.23,129.50,23.18,78,100,3.31,CN,1605847913
...,...,...,...,...,...,...,...,...,...
590,Yangjiang,21.85,111.97,87.01,57,7,7.18,CN,1605848033
591,Kapit,2.02,112.93,93.85,52,93,1.01,MY,1605848033
592,Khorinsk,52.17,109.77,13.39,86,25,13.38,RU,1605848033
593,Tura,25.52,90.22,74.23,74,18,0.92,IN,1605847997


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = city_df[["Lat","Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)

In [6]:


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Drop rows with NaN values
city_df_clean = city_df.dropna()

# Max temperature lower than 80 but higher than 70
temp70 = city_df_clean["Max Temp"] > 70
temp70 = city_df_clean.loc[temp70]
temp80 = temp70["Max Temp"] < 80
nice_weather = temp70.loc[temp80]
nice_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ibrā’,22.69,58.53,75.09,43,68,7.36,OM,1605847912
1,Kirakira,-10.45,161.92,79.79,86,100,10.78,SB,1605847679
3,Kapaa,22.08,-159.32,77.00,78,40,14.99,US,1605847913
7,Gamba,-2.65,10.00,75.97,92,100,4.76,GA,1605847915
10,Mogadishu,2.04,45.34,78.80,74,40,9.17,SO,1605847819
...,...,...,...,...,...,...,...,...,...
577,Álamos,27.02,-108.93,75.00,59,0,3.51,MX,1605848030
579,Qui Nhon,13.77,109.23,79.34,80,45,6.51,VN,1605847785
582,Harper,4.38,-7.72,78.44,82,80,5.84,LR,1605848031
586,Muroto-misakicho,33.28,134.15,75.20,83,75,2.24,JP,1605848032


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotels,Hotel Name
0,Ibrā’,22.69,58.53,75.09,43,68,7.36,OM,1605847912,,
1,Kirakira,-10.45,161.92,79.79,86,100,10.78,SB,1605847679,,
3,Kapaa,22.08,-159.32,77,78,40,14.99,US,1605847913,,
7,Gamba,-2.65,10,75.97,92,100,4.76,GA,1605847915,,
10,Mogadishu,2.04,45.34,78.8,74,40,9.17,SO,1605847819,,
...,...,...,...,...,...,...,...,...,...,...,...
579,Qui Nhon,13.77,109.23,79.34,80,45,6.51,VN,1605847785,,
582,Harper,4.38,-7.72,78.44,82,80,5.84,LR,1605848031,,
586,Muroto-misakicho,33.28,134.15,75.2,83,75,2.24,JP,1605848032,,
593,Tura,25.52,90.22,74.23,74,18,0.92,IN,1605847997,,


In [56]:
city_lat = nice_weather["Lat"]
city_lng = nice_weather["Lng"]


target_search = "hotel"
target_radius = 5000
target_type = "lodging"

hotels_list = []

# Parameters dictionary
params = {
#     "location": lat_lng,
    "keyword": target_search,
    "radius" : target_radius,
    "type": target_type,
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
query_url = f'location=37.7749%2C-122.4194&keyword={target_search}&radius={target_radius}&type={target_type}&key={g_key}'


for hotel in city_lat:
    
    response = requests.get(base_url, params=params).json()
    


In [57]:
response
    

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [22]:
# Wind speed less than 10 mph
low_wind = nice_weather["Wind Speed"] < 10
nice_weather = nice_weather.loc[low_wind]


# Zero cloudiness
zero_clouds = nice_weather["Cloudiness"] == 0
nice_weather = nice_weather.loc[zero_clouds]
hotel_df = nice_weather
nice_weather[zero_clouds]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Valle de Allende,26.93,-105.40,71.76,38,0,6.89,MX,1605810372
50,Riyadh,24.69,46.72,71.60,43,0,3.36,SA,1605810378
89,Mathbaria,22.29,89.96,71.73,85,0,3.22,BD,1605810387
91,Ancud,-41.87,-73.82,75.99,47,0,8.05,CL,1605810387
129,Hambantota,6.12,81.12,75.20,100,0,4.70,LK,1605810395
131,Aswān,24.09,32.91,71.37,27,0,9.04,EG,1605810396
184,Ban Nahin,18.24,104.21,73.92,82,0,2.53,LA,1605810407
273,Qui Nhon,13.77,109.23,74.14,89,0,4.18,VN,1605810428
303,Hong Kong,22.29,114.16,77.00,88,0,7.02,HK,1605810435
316,Guerrero Negro,27.98,-114.06,76.42,47,0,8.28,MX,1605810438


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
